# Insert Data BigQuery

## Libs

In [ ]:
import os
import logging
from google.cloud import bigquery
from google.cloud import storage
import pandas as pd
from typing import List, Optional


## Configurar cliente

In [11]:
# Configurando o caminho para o arquivo de credenciais
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../../key.json"

# Configurar cliente BigQuery e Storage
client = bigquery.Client()
gsclient = storage.Client()


In [7]:
def load_bigquery(
    query: str,
    project_id: str = "bankmarketingdatapipeline",
    name_view: Optional[str] = None
) -> pd.DataFrame:
    """
    Executa uma consulta no BigQuery e retorna os resultados como um DataFrame.

    Args:
        query (str): Consulta SQL a ser executada.
        project_id (str): ID do projeto do BigQuery.
        name_view (Optional[str]): Nome da visualização a ser criada (opcional).
    
    Returns:
        pd.DataFrame: Resultados da consulta como um DataFrame.
    """
    client = bigquery.Client(project=project_id)
    
    if name_view:
        # Cria uma visualização no BigQuery, se necessário
        view_query = f"""
        CREATE OR REPLACE VIEW `{project_id}.{name_view}` AS {query}
        """
        client.query(view_query).result()  # Executa a criação da visualização
        print(f"View '{name_view}' criada com sucesso.")
    
    # Executa a consulta e retorna um DataFrame
    query_job = client.query(query)
    df = query_job.to_dataframe()
    return df

## Criação e Ingestão da tabela trusted_order

In [82]:
# Carregar a tabela
job_config = bigquery.LoadJobConfig(
    schema=[
        bigquery.SchemaField(
            "order_id", 
            "INTEGER", 
            mode="REQUIRED", 
            description="ID único do pedido"
        ),
        bigquery.SchemaField(
            "order_date", 
            "TIMESTAMP", 
            mode="REQUIRED", 
            description="Data e hora em que o pedido foi realizado"
        ),
        bigquery.SchemaField(
            "order_customer_id", 
            "INTEGER", 
            mode="REQUIRED", 
            description="ID do cliente que fez o pedido"
        ),
        bigquery.SchemaField(
            "order_status", 
            "STRING", 
            mode="REQUIRED", 
            description="Status atual do pedido (ex: 'em processamento', 'enviado')"
        )
    ]
)

uri = "gs://raw_retail/orders/part-00000"
table_id = f"{project_id}.db_retail.trusted_order"

# Carregar a tabela
load_job = client.load_table_from_uri(
    uri, table_id, job_config=job_config
)  
load_job.result()  # Aguardar a conclusão do job

# Obter a tabela
table = client.get_table(table_id)

# Atualizar descrição da tabela
table.description = "Tabela contendo os pedidos confiáveis do banco de dados de varejo."
table = client.update_table(table, ["description"])